### Import Libraries

In [32]:
import pandas as pd
import requests

In [33]:
pd.set_option('display.max_columns', 30)

### Helper Functions

In [34]:
def get_json_from_url(url, headers=None, timeout=10):
    response = requests.get(url, headers=headers,timeout=timeout)
    response.raise_for_status() 
    
    return response.json()

### Read Stations Info

In [35]:
stations_info_data = get_json_from_url("https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information")

In [36]:
stations_info = stations_info_data.get('data', {}).get('stations', [])

In [37]:
len(stations_info)

863

In [38]:
stations_info_df = pd.DataFrame(stations_info)

In [39]:
stations_info_df.head()

,station_id,name,physical_configuration,lat,lon,altitude,address,capacity,is_charging_station,rental_methods,groups,obcn,short_name,nearby_distance,_ride_code_support,rental_uris,post_code,is_valet_station,cross_street
0,7000,Fort York Blvd / Capreol Ct,REGULAR,43.639832,-79.395954,NaN,Fort York Blvd / Capreol Ct,47,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[South, Fort York - Entertainment District]",647-643-9607,647-643-9607,500.0,True,{},NaN,NaN,NaN
1,7001,Wellesley Station Green P,ELECTRICBIKESTATION,43.664964,-79.383550,NaN,Yonge / Wellesley,23,True,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[E-Charging , South, Church Wellesley / Yorkvi...",416-617-9576,416-617-9576,500.0,True,{},M4Y 1G7,NaN,NaN
2,7002,St. George St / Bloor St W,REGULAR,43.667131,-79.399555,NaN,St. George St / Bloor St W,17,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[South, U of T - Hospital Row]",647-643-9615,647-643-9615,500.0,True,{},NaN,NaN,NaN
3,7003,Madison Ave / Bloor St W,REGULAR,43.667018,-79.402796,NaN,Madison Ave / Bloor St W,15,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[South, Bloor St W / Annex]",647-631-4587,647-631-4587,500.0,True,{},NaN,NaN,NaN
4,7005,King St W / York St,REGULAR,43.648001,-79.383177,0.0,King St W / York St,21,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[South, Financial District]",647-643-9693,647-643-9693,500.0,True,{},NaN,NaN,NaN


In [40]:
len(stations_info_df)

863

### Read Stations Status

In [41]:
stations_status_data = get_json_from_url("https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_status")

In [42]:
stations_status = stations_status_data.get('data', {}).get('stations', [])

In [43]:
len(stations_status)

863

In [44]:
stations_status_df = pd.DataFrame(stations_status)

In [45]:
stations_status_df.head()

,station_id,num_bikes_available,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,traffic
0,7000,22,"{'mechanical': 21, 'ebike': 1}",0,24,1,1739707008,False,IN_SERVICE,1,1,1,None
1,7001,7,"{'mechanical': 6, 'ebike': 1}",0,16,0,1739706953,True,IN_SERVICE,1,1,1,None
2,7002,2,"{'mechanical': 2, 'ebike': 0}",0,14,1,1739706960,False,IN_SERVICE,1,1,1,None
3,7003,1,"{'mechanical': 1, 'ebike': 0}",0,14,0,1739707036,False,IN_SERVICE,1,1,1,None
4,7005,15,"{'mechanical': 15, 'ebike': 0}",0,6,0,1739707016,False,IN_SERVICE,1,1,1,None


### Data Transformation

#### 1. Join Stations Info and Stations Status 

In [46]:
merged_df = pd.merge(stations_info_df, stations_status_df, on="station_id", how="inner")

In [47]:
merged_df.head()

,station_id,name,physical_configuration,lat,lon,altitude,address,capacity,is_charging_station_x,rental_methods,groups,obcn,short_name,nearby_distance,_ride_code_support,...,post_code,is_valet_station,cross_street,num_bikes_available,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,last_reported,is_charging_station_y,status,is_installed,is_renting,is_returning,traffic
0,7000,Fort York Blvd / Capreol Ct,REGULAR,43.639832,-79.395954,NaN,Fort York Blvd / Capreol Ct,47,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[South, Fort York - Entertainment District]",647-643-9607,647-643-9607,500.0,True,...,NaN,NaN,NaN,22,"{'mechanical': 21, 'ebike': 1}",0,24,1,1739707008,False,IN_SERVICE,1,1,1,None
1,7001,Wellesley Station Green P,ELECTRICBIKESTATION,43.664964,-79.383550,NaN,Yonge / Wellesley,23,True,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[E-Charging , South, Church Wellesley / Yorkvi...",416-617-9576,416-617-9576,500.0,True,...,M4Y 1G7,NaN,NaN,7,"{'mechanical': 6, 'ebike': 1}",0,16,0,1739706953,True,IN_SERVICE,1,1,1,None
2,7002,St. George St / Bloor St W,REGULAR,43.667131,-79.399555,NaN,St. George St / Bloor St W,17,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[South, U of T - Hospital Row]",647-643-9615,647-643-9615,500.0,True,...,NaN,NaN,NaN,2,"{'mechanical': 2, 'ebike': 0}",0,14,1,1739706960,False,IN_SERVICE,1,1,1,None
3,7003,Madison Ave / Bloor St W,REGULAR,43.667018,-79.402796,NaN,Madison Ave / Bloor St W,15,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[South, Bloor St W / Annex]",647-631-4587,647-631-4587,500.0,True,...,NaN,NaN,NaN,1,"{'mechanical': 1, 'ebike': 0}",0,14,0,1739707036,False,IN_SERVICE,1,1,1,None
4,7005,King St W / York St,REGULAR,43.648001,-79.383177,0.0,King St W / York St,21,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[South, Financial District]",647-643-9693,647-643-9693,500.0,True,...,NaN,NaN,NaN,15,"{'mechanical': 15, 'ebike': 0}",0,6,0,1739707016,False,IN_SERVICE,1,1,1,None


In [48]:
len(merged_df)

863

#### 2. Remove stations not in service

In [49]:
print(merged_df['status'].unique())

['IN_SERVICE']


In [50]:
filtered_df = merged_df[merged_df['status'] == 'IN_SERVICE']

In [51]:
len(filtered_df)

863

#### 3. Apply Transformations

In [52]:
filtered_df.dtypes

station_id                    object
name                          object
physical_configuration        object
lat                          float64
lon                          float64
altitude                     float64
address                       object
capacity                       int64
is_charging_station_x           bool
rental_methods                object
groups                        object
obcn                          object
short_name                    object
nearby_distance              float64
_ride_code_support              bool
rental_uris                   object
post_code                     object
is_valet_station              object
cross_street                  object
num_bikes_available            int64
num_bikes_available_types     object
num_bikes_disabled             int64
num_docks_available            int64
num_docks_disabled             int64
last_reported                  int64
is_charging_station_y           bool
status                        object
i

In [53]:
# Add last_updated column from station_status API response

last_updated = stations_status_data['last_updated']
filtered_df['last_updated'] = pd.to_datetime(last_updated, unit='s')
filtered_df.head()

,station_id,name,physical_configuration,lat,lon,altitude,address,capacity,is_charging_station_x,rental_methods,groups,obcn,short_name,nearby_distance,_ride_code_support,...,is_valet_station,cross_street,num_bikes_available,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,last_reported,is_charging_station_y,status,is_installed,is_renting,is_returning,traffic,last_updated
0,7000,Fort York Blvd / Capreol Ct,REGULAR,43.639832,-79.395954,NaN,Fort York Blvd / Capreol Ct,47,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[South, Fort York - Entertainment District]",647-643-9607,647-643-9607,500.0,True,...,NaN,NaN,22,"{'mechanical': 21, 'ebike': 1}",0,24,1,1739707008,False,IN_SERVICE,1,1,1,None,2025-02-16 11:58:52
1,7001,Wellesley Station Green P,ELECTRICBIKESTATION,43.664964,-79.383550,NaN,Yonge / Wellesley,23,True,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[E-Charging , South, Church Wellesley / Yorkvi...",416-617-9576,416-617-9576,500.0,True,...,NaN,NaN,7,"{'mechanical': 6, 'ebike': 1}",0,16,0,1739706953,True,IN_SERVICE,1,1,1,None,2025-02-16 11:58:52
2,7002,St. George St / Bloor St W,REGULAR,43.667131,-79.399555,NaN,St. George St / Bloor St W,17,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[South, U of T - Hospital Row]",647-643-9615,647-643-9615,500.0,True,...,NaN,NaN,2,"{'mechanical': 2, 'ebike': 0}",0,14,1,1739706960,False,IN_SERVICE,1,1,1,None,2025-02-16 11:58:52
3,7003,Madison Ave / Bloor St W,REGULAR,43.667018,-79.402796,NaN,Madison Ave / Bloor St W,15,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[South, Bloor St W / Annex]",647-631-4587,647-631-4587,500.0,True,...,NaN,NaN,1,"{'mechanical': 1, 'ebike': 0}",0,14,0,1739707036,False,IN_SERVICE,1,1,1,None,2025-02-16 11:58:52
4,7005,King St W / York St,REGULAR,43.648001,-79.383177,0.0,King St W / York St,21,False,"[KEY, TRANSITCARD, CREDITCARD, PHONE]","[South, Financial District]",647-643-9693,647-643-9693,500.0,True,...,NaN,NaN,15,"{'mechanical': 15, 'ebike': 0}",0,6,0,1739707016,False,IN_SERVICE,1,1,1,None,2025-02-16 11:58:52


In [54]:
# Create mechanical_bikes_available and electric_bikes_available features

def extract_mechanical_bikes(bike_types): 
    return bike_types.get('mechanical', 0) 

def extract_electric_bikes(bike_types):
    return bike_types.get('ebike', 0) 

filtered_df['mechanical_bikes_available'] = filtered_df['num_bikes_available_types'].apply(extract_mechanical_bikes)
filtered_df['electric_bikes_available'] = filtered_df['num_bikes_available_types'].apply(extract_electric_bikes)


In [55]:
# Convert is_charging_station column to Binary
filtered_df['is_charging_station']  = filtered_df['is_charging_station_x'].map({True: 1, False: 0})

In [56]:
# Convert rental_methods and groups to comma separated text keywords.

filtered_df['rental_methods'] = filtered_df['rental_methods'].apply(lambda data: ",".join([f"'{keyword}'" for keyword in data]))
filtered_df['groups'] = filtered_df['groups'].apply(lambda data: ",".join([f"'{keyword}'" for keyword in data]))

In [57]:
# Select columns for target csv
df_cleaned = filtered_df[[
    'last_updated', 'station_id', 'name', 'physical_configuration', 
    'lat', 'lon', 'altitude',
    'address', 'capacity', 'rental_methods', 'groups', 'obcn', 'nearby_distance', 
    'num_bikes_available', 'mechanical_bikes_available', 'electric_bikes_available',
    'num_bikes_disabled', 'num_docks_available', 'num_docks_disabled', 
    'is_installed', 'is_renting', 'is_returning', 'last_reported', 'is_charging_station', 'status'
]]


In [58]:
# Rename columns for target csv
df_cleaned.columns = [
    'last_update', 'station_id', 'station_name', 'physical_configuration', 
    'latitude', 'longitude', 'altitude', 
    'address', 'capacity', 'rental_methods', 'groups', 'obcn', 'nearby_distance', 
    'num_bikes_available', 'mechanical_bikes_available', 'electric_bikes_available', 
    'num_bikes_disabled', 'num_docks_available', 'num_docks_disabled', 
    'is_installed', 'is_renting', 'is_returning', 'last_reported', 'is_charging_station', 'status'
]

In [59]:
df_cleaned.head()

,last_update,station_id,station_name,physical_configuration,latitude,longitude,altitude,address,capacity,rental_methods,groups,obcn,nearby_distance,num_bikes_available,mechanical_bikes_available,electric_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,is_installed,is_renting,is_returning,last_reported,is_charging_station,status
0,2025-02-16 11:58:52,7000,Fort York Blvd / Capreol Ct,REGULAR,43.639832,-79.395954,NaN,Fort York Blvd / Capreol Ct,47,"'KEY','TRANSITCARD','CREDITCARD','PHONE'","'South','Fort York - Entertainment District'",647-643-9607,500.0,22,21,1,0,24,1,1,1,1,1739707008,0,IN_SERVICE
1,2025-02-16 11:58:52,7001,Wellesley Station Green P,ELECTRICBIKESTATION,43.664964,-79.383550,NaN,Yonge / Wellesley,23,"'KEY','TRANSITCARD','CREDITCARD','PHONE'","'E-Charging ','South','Church Wellesley / York...",416-617-9576,500.0,7,6,1,0,16,0,1,1,1,1739706953,1,IN_SERVICE
2,2025-02-16 11:58:52,7002,St. George St / Bloor St W,REGULAR,43.667131,-79.399555,NaN,St. George St / Bloor St W,17,"'KEY','TRANSITCARD','CREDITCARD','PHONE'","'South','U of T - Hospital Row'",647-643-9615,500.0,2,2,0,0,14,1,1,1,1,1739706960,0,IN_SERVICE
3,2025-02-16 11:58:52,7003,Madison Ave / Bloor St W,REGULAR,43.667018,-79.402796,NaN,Madison Ave / Bloor St W,15,"'KEY','TRANSITCARD','CREDITCARD','PHONE'","'South','Bloor St W / Annex'",647-631-4587,500.0,1,1,0,0,14,0,1,1,1,1739707036,0,IN_SERVICE
4,2025-02-16 11:58:52,7005,King St W / York St,REGULAR,43.648001,-79.383177,0.0,King St W / York St,21,"'KEY','TRANSITCARD','CREDITCARD','PHONE'","'South','Financial District'",647-643-9693,500.0,15,15,0,0,6,0,1,1,1,1739707016,0,IN_SERVICE


In [61]:
len(df_cleaned)

863

### Write to CSV

In [60]:
df_cleaned.to_csv('stations.csv', index=False)